In [7]:
from decouple import config

In [8]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [9]:
ALPHA_VANTAGE_API_KEY=config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [10]:
import os
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode
import requests


@dataclass
class AlphaVantageAPIClient:
    ticker: str = "AAPL"
    function: Literal["TIME_SERIES_INTRADAY"] = "TIME_SERIES_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "1min"
    api_key: str = os.environ.get("ALPHA_VANTAGE_API_KEY", None)

    def get_api_key(self):
        """Retrieve the API key."""
        if not self.api_key:
            raise ValueError("API key is required. Set it via environment variables or pass it directly.")
        return self.api_key

    def get_params(self):
        """Generate query parameters."""
        return {
            "symbol": self.ticker,
            "function": self.function,
            "interval": self.interval,
            "apikey": self.get_api_key(),
        }

    def generate_url(self):
        """Construct the full API URL."""
        base_url = "https://www.alphavantage.co/query"
        params = self.get_params()
        encoded_params = urlencode(params)
        return f"{base_url}?{encoded_params}"

    def fetch_data(self):
        """Fetch data from Alpha Vantage API."""
        url = self.generate_url()
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for non-200 responses
        return response.json()

    def get_stock_data(self):
        """Retrieve and transform stock data."""
        data = self.fetch_data()
        dataset_key = [x for x in data.keys() if x.lower() != "meta data"][0]
        results = data[dataset_key]
        dataset = []
        for timestamp_str, values in results.items():
            dataset.append({"timestamp": timestamp_str, **values})
        return dataset


In [11]:
stock_api_client = AlphaVantageAPIClient()
dataset = stock_api_client.get_stock_data()
dataset

[{'timestamp': '2024-11-29 16:59:00',
  '1. open': '237.3600',
  '2. high': '237.4900',
  '3. low': '237.3600',
  '4. close': '237.4450',
  '5. volume': '616'},
 {'timestamp': '2024-11-29 16:58:00',
  '1. open': '237.3750',
  '2. high': '237.4900',
  '3. low': '237.3535',
  '4. close': '237.3600',
  '5. volume': '598'},
 {'timestamp': '2024-11-29 16:57:00',
  '1. open': '237.3900',
  '2. high': '237.3987',
  '3. low': '237.3000',
  '4. close': '237.3750',
  '5. volume': '339'},
 {'timestamp': '2024-11-29 16:56:00',
  '1. open': '237.3250',
  '2. high': '237.3900',
  '3. low': '237.3000',
  '4. close': '237.3000',
  '5. volume': '5383'},
 {'timestamp': '2024-11-29 16:55:00',
  '1. open': '237.3700',
  '2. high': '237.3700',
  '3. low': '237.3000',
  '4. close': '237.3300',
  '5. volume': '43'},
 {'timestamp': '2024-11-29 16:54:00',
  '1. open': '237.4000',
  '2. high': '237.4000',
  '3. low': '237.3000',
  '4. close': '237.3250',
  '5. volume': '1252'},
 {'timestamp': '2024-11-29 16:53: